In [ ]:
from scipy.stats import beta as beta_dist
from plotting_utils import plot_gaussian_hypothesis_test
from plotting_utils import plot_type_ii_error_analysis, plot_beta_prior_comparison, plot_prior_vs_posterior
from plotting_utils import plot_informative_prior_posterior_comparison, plot_weakly_informative_prior_with_variants
from plotting_utils import plot_multiple_posteriors_comparison

<style>
.reveal h1 { font-size: 1.8em; color: #4a90d9; }
.reveal h2 { font-size: 1.4em; color: #4a90d9; }
.reveal h3 { font-size: 1.1em; color: #4a90d9; }
.reveal { color: #333; }
</style>

# From Manual Bayesian A/B Tests to Continuous Thompson Sampling

**A Post Mortem of our Passkeys A/B tests and a proposal for automated approaches**

<br><br><br><br>

<p style="font-size: 0.6em; color: #666; line-height: 1.6;">
All the notebooks with running code can be cloned from
<br><a href="https://github.com/alaindemour/ABtest_methodologies" style="color: #4a90d9;">https://github.com/alaindemour/ABtest_methodologies</a>
<br>Created with the help of Claude Sonnet 4.6
</p>

# Presentation Overview

1. **Executive Summary** Business Problem and Solution
2. **Classical vs. Bayesian Statistics**
3. **Why Bayesian is Better** for CX / pricing A/B tests
4. **Non-Inferiority Test** — NHST fails, Bayesian succeeds
5. **Select Best Variant** — direct probability
6. **Corporate Iteration Speed** — the deployment bottleneck
7. **Thompson Sampling** — the fully automated solution

# Reminder: Our Experiment Setup
- **Variant A**: Combo password+passkey page with passkey creation as a default on 🎯
- **Variant B**: Combo password_passkey page with passkey creation as a default off
- **Variant C**: Multi Pages, password completely separate from passkeys, with step by step instructions
- **Control Group**: Legacy flow with no passkey creation offered

For Amazon linking the KPI was **maximum number of passkey creations** to improve gating rate while **avoiding any degradation** of the existing
linking rate

Not the purpose of the this presntation but A significanlty better for true end to end sucess ratio, C better for the passkey creation step in isolation.

# Business Problem
We would like to **experiment with CX often**, test **a large number of variants**, get results **within weeks** and **iterate quickly** (e.g. "fail early")

## Experiment Design Reality
Product, engineering, DCE, legal **haggled for months** on design and copy to get their **preferred** ideas in the **only 3** variants we could realistically launch, variants were **stakeholder compromises** to fit 3 options, not what we really wanted to test. That's a reality of the corporate world we have to accept.
 
## Real-World Timeline

| Date | Event |
|------|-------|
| **Oct** | Bug at launch, variant C gets all the traffic, missing control group |
| **Early Nov** | Variant A wins with >90% probability — we know |
| **Mid Nov** | Amazon release freeze (Black Friday) |
| **Late Nov** | Release issues, more rollbacks |
| **December** | Digital Channel release freeze, holiday delays |
| **January** | Winner (variant A) finally ramped up |

**~3 months** from "we know" to "users benefit" and next iterations

# Reframing of the Problem: The Multi-Armed Bandit Framework
One of the most famous problems in computer science (and life): the **Dilemma** — Explore (try machines) vs. Exploit (play the best)

<div style="display: flex; align-items: center; gap: 40px;">
<div style="flex: 1;">

**Casino analogy**: K slot machines, unknown payouts, limited budget.

| Casino | A/B Testing |
|--------|-------------|
| Slot machines | Variants |
| Pull lever | Show variant |
| Payout | User converts |


</div>
<div style="flex: 0 0 auto; text-align: center;">
<img src="images/multi_armed_bandit.jpg" style="max-height: 320px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.15);">
</div>
</div>

## An Optimal Solution: Thompson Sampling

- Can run a **massive number of variants** in parallel
- Completely algorithmic and adaptive: **no release process** or anything manual involved
- Mathematically proven to be **optimal**: converges the fastest to the best variant
- Allows dynamic addition or removal of any number of variants, **can be always on**

# Part 2: Classical vs. Bayesian Statistics

Caveat: once you have millions of data points **none of this matters**; statistical inference is useful when you need to make decisions based on small samples and iterate quickly

## The Fundamental Questions

Classical statistics a.k.a. Frequentist uses a methodology called Null Hypothesis Significance Testing (NHST). If you hear **p-values** or **confidence intervals**, **Null Hypothesis** or **this experiment is underpowered**, it is NHST. Bayesian approaches directly compute precise probabilities for **all the possible hypotheses**, not just a single $H_0$.

| Framework | Question | Notation |
|-----------|----------|----------|
| **NHST** | "How likely is my observation (a.k.a. sample a.k.a data), assuming $H_0$?" | p(data \| Hypothesis) |
| **Bayesian** | "What is the precise probability of all hypotheses, given the observation?" | p(Hypothesis \| data) |

## NHST in a Nutshell

1. Assume a so called **Null Hypothesis** $H_0$ which is typically what you don't want to be true e.g. **the new drug has no effect** or **the new passkey creation UX degrades conversion**
2. Compute a "test statistic" from the observed data; in this presentation it is a mean but can be any statistical measure
3. If result is "unlikely enough" (p-value < 0.05), reject $H_0$

### Key Caveats

- Rejecting $H_0$ does **not** prove the alternative
- The 5% threshold is a **convention**, not a law of nature
- No quantified probability of being correct — just "unlikely" or "not unlikely"
- The math involved requires large samples for significance
- The math does not "compose", **it cannot implement the Thompson Sampling Algorithm which is iterative**

## Bayesian in a Nutshell

1. Pick a **prior belief** distribution for the measure we are interested in (conventionally noted as $\theta$ and also called "the parameter"), the parameter is how we represent the various hypotheses e.g. $\theta$ = conversion rate
2. Observe data
3. Apply **Bayes' theorem** → get **posterior belief**

$$\boxed{P(\theta \mid \text{data}) = \frac{P(\text{data} \mid \theta) \cdot P(\theta)}{P(\text{data})}}$$

✅ 4. The Posterior belief can become the prior belief for the next update, that the **key property** of the Bayesian update which support Thompson sampling

Full probability distribution — compute anything you need at any time during the experiment:
point estimates, credible intervals, P(better than threshold), expected values

In [ ]:
# What is a Probability Distribution?
from scipy.stats import beta as beta_dist
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))

# Beta(9, 4): mean ≈ 0.69 — same center, but wider spread than a large-sample posterior
a, b = 9, 4
x = np.linspace(0, 1, 500)
y = beta_dist.pdf(x, a, b)

# Full distribution curve over the complete [0, 1] domain
ax.plot(x, y, 'k-', lw=2.5)

# Shade the interval [0.70, 0.80]
lo, hi = 0.70, 0.80
mask = (x >= lo) & (x <= hi)
prob = beta_dist.cdf(hi, a, b) - beta_dist.cdf(lo, a, b)
ax.fill_between(x[mask], y[mask], alpha=0.35, color='#4a90d9',
                label=f'P({lo:.0%} < conversion rate < {hi:.0%}) = {prob:.1%}')

ax.set_xlim(0, 1)
ax.set_xlabel('Conversion Rate', fontsize=13)
ax.set_ylabel('Probability Density', fontsize=13)
ax.set_title('What is a Probability Distribution?\n'
             'The shaded area = probability that the true conversion rate is in [70%, 80%]',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()


## Some Math: The Beta Probability Distribution

<div style="display: flex; align-items: center; gap: 40px;">
<div style="flex: 1;">

- Models a **probability** (values in [0, 1])
- Controlled by two parameters α, β
- **α − 1** = prior successes, **β − 1** = prior failures
- As data accumulates, the distribution **narrows** (more certainty)
- Perfect for modeling conversion rates

$$\text{Beta}(x; \alpha, \beta) = \frac{x^{\alpha-1}(1-x)^{\beta-1}}{\int_0^1 t^{\alpha-1}(1-t)^{\beta-1}\,dt}$$

</div>
<div style="flex: 0 0 auto; text-align: center;">
<img src="images/PDF_of_the_Beta_distribution.gif" style="max-height: 320px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.15);" alt="Beta distribution animation">
</div>
</div>

In [ ]:
# Plot comparison of different Beta prior distributions
fig, axes = plot_beta_prior_comparison()

In [ ]:
# Bayesian Update: weakly informative prior → posterior after moderate data
# Prior: Beta(2, 2) — weakly informative, spread across full [0,1]
# Data: 12 successes in 20 trials (60% rate)
# Posterior: Beta(14, 10) — shifted toward 0.6 but still broad
alpha_prior, beta_prior = 2, 2
n_obs, k_obs = 20, 12
alpha_post = alpha_prior + k_obs
beta_post = beta_prior + (n_obs - k_obs)

fig, ax, _, _ = plot_informative_prior_posterior_comparison(
    alpha_prior=alpha_prior,
    beta_prior=beta_prior,
    alpha_posterior=alpha_post,
    beta_posterior=beta_post,
    x_limits=(0, 1)
)
print(f"Prior: Beta({alpha_prior},{beta_prior}) — mean={alpha_prior/(alpha_prior+beta_prior):.0%}")
print(f"Data: {k_obs} successes in {n_obs} trials")
print(f"Posterior: Beta({alpha_post},{beta_post}) — mean={alpha_post/(alpha_post+beta_post):.0%}")

# Part 3: Why Bayesian is Better

## NHST Problems for Product A/B Tests

| Problem | Impact |
|---|---|
| **Small samples** | At launch, n=150 per variant. NHST: "inconclusive." |
| **p-value hacking** | Can't peek at results without inflating false positives |
| **Unbalanced samples** | 7,000 control vs. 150 variant — NHST loses power |
| **Binary output** | "Reject" or "fail to reject" — no probabilities |

## Where Bayesian Excels

| Advantage | Detail |
|---|---|
| **Small samples** | Prior knowledge + data = meaningful conclusions |
| **Unbalanced allocation** | Each variant analyzed independently |
| **Many variants** | No multiple comparison penalties |
| **Actionable output** | "47% chance B is best" — not just reject/fail |
| **Continuous monitoring** | Check anytime, no p-hacking concerns |

## Fun Fact: FDA Embraces Bayesian (January 2026)

> The FDA issued draft guidance: **"Use of Bayesian Methodology in Clinical Trials of Drugs and Biological Products"**

https://www.fda.gov/regulatory-information/search-fda-guidance-documents/use-bayesian-methodology-clinical-trials-drug-and-biological-products

If the most conservative regulator in the world is adopting Bayesian methods, the case for product A/B testing is even stronger.

# Part 4: Non-Inferiority Test for Amazon Linking

## The Setup

- Existing flow: **~71% completion rate**
- Adding passkey creation (extra pages/clicks)
- **Question**: Does the new experience cause unacceptable degradation?
- **Non-inferiority margin**: $\epsilon$ = 2%

| Group | Sample size | Conversion rate |
|-------|-----------|----------------|
| Control | 32,106 | 70.9% |
| Variant A | 4,625 | 70.1% |
| Variant B | 2,100 | 68.2% |
| Variant C | 2,022 | 69.0% |

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, beta as beta_dist

# --- Experiment Data ---
nC = 32106
xC = 22772
control_rate = xC / nC

variants = {
    'A': {'n': 4625, 'x': 3244},
    'B': {'n': 2100, 'x': 1433},
    'C': {'n': 2022, 'x': 1396}
}

epsilon = 0.02  # 2% non-inferiority margin

print(f"Control: n={nC:,}, rate={control_rate:.2%}")
for name, d in variants.items():
    r = d['x'] / d['n']
    print(f"Variant {name}: n={d['n']:,}, rate={r:.2%}")
print(f"\nNon-inferiority margin (epsilon): {epsilon:.0%}")
print(f"Non-inferiority threshold: {control_rate - epsilon:.2%}")

## NHST Result: Inconclusive

**p-value ~ 0.45** (45%) — very far from the 5% threshold

NHST says: "We can't say anything."

The test is **severely underpowered** with a sample of n=2,022.

In [ ]:
# NHST non-inferiority test on Variant C
nX = variants['C']['n']
xX = variants['C']['x']
hatpC = xC / nC
hatpA = xX / nX
hatDelta = hatpA - hatpC

# Unpooled SE (appropriate for non-inferiority)
SE = np.sqrt(hatpC * (1 - hatpC) / nC + hatpA * (1 - hatpA) / nX)
mu_H0 = -epsilon

# p-value: right tail P(Delta >= observed | H0)
p_value = norm.sf(hatDelta, loc=mu_H0, scale=SE)

# Power analysis
pooled_p = (xC + xX) / (nC + nX)
SE_H1 = np.sqrt(pooled_p * (1 - pooled_p) * (1/nC + 1/nX))
critical_value = norm.isf(0.05, loc=mu_H0, scale=SE)
power = 1 - norm.cdf(critical_value, loc=0, scale=SE_H1)

print("NHST Non-Inferiority Test (Variant C)")
print("=" * 50)
print(f"Observed difference: {hatDelta:.4f}")
print(f"Standard error: {SE:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Power: {power:.1%}")
print()
if p_value <= 0.05:
    print("Result: REJECT H0 — non-inferiority established")
else:
    print("Result: FAIL TO REJECT — inconclusive")
    print(f"  (Power is only {power:.1%} — test is severely underpowered)")
    print(f"  NHST cannot help us with n={nX} for this variant.")

## Bayesian: Actionable Answer

<!-- **Weakly informative prior** centered on expected performance:

$$\text{Beta}(\alpha_0, \beta_0) + \text{Data} \Rightarrow \text{Beta}(\alpha_0 + k, \; \beta_0 + n - k)$$

Then directly compute: $P(\text{variant} > \text{control} - \epsilon)$ -->

In [ ]:
from bayesian import test_non_inferiority_weakly_informative
from plotting_utils import plot_weakly_informative_prior_with_variants

# Run Bayesian non-inferiority test on all variants
expected_degradation = 0.01  # Domain knowledge: adding clicks may degrade by ~1%

results_ni = test_non_inferiority_weakly_informative(
    n_control=nC,
    x_control=xC,
    variants_data=variants,
    epsilon=epsilon,
    expected_degradation=expected_degradation,
    alpha_prior_strength=20,  # Weak prior (high entropy)
    threshold=0.95
)

print("Bayesian Non-Inferiority Test Results")
print("=" * 60)
print(f"Prior centered at: {control_rate - expected_degradation:.2%}")
print(f"Test threshold: {control_rate - epsilon:.2%}")
print()


In [ ]:

for name, res in results_ni.items():
    status = "NON-INFERIOR" if res['is_non_inferior'] else "NOT NON-INFERIOR"
    observed_rate = variants[name]['x'] / variants[name]['n']
    print(f"Variant {name}: {status}")
    print(f"  Observed rate: {observed_rate:.2%}, Posterior mean: {res['variant_rate']:.2%}")
    print(f"  P(variant > threshold): {res['probability']:.2%}")
    print()


In [ ]:

# Visualize
fig, ax = plot_weakly_informative_prior_with_variants(results_ni)

## Takeaway: Same Data, Different Answers

| Method | Variant C Result | Actionable? |
|--------|-----------------|-------------|
| **NHST** | p-value ~ 0.45 (45%), "fail to reject" | No |
| **Bayesian** | P(non-inferior) > 95% | **Yes** |

# Part 5: Select Best Variant

## NHST Can't Answer "Which is Best?"

| Approach | Problem |
|---|---|
| Highest observed rate | Ignores uncertainty |
| Pairwise t-tests + Bonferroni | Very conservative |
| ANOVA + post-hoc | Only says "something differs" |

**None directly answer**: "What is P(A is best)?"

## Bayesian: Direct Probability

1. Compute **posterior Beta** for each variant
2. **Monte Carlo**: draw 100k samples from each
3. Count how often each variant wins

Result: $P(A \text{ is best}) = 88\%$, $P(B) = 3\%$, $P(C) = 9\%$

No corrections needed. Scales to any number of variants.

In [ ]:
from bayesian import select_best_variant
from plotting_utils import plot_multiple_posteriors_comparison

# Select the best variant using Monte Carlo simulation
selection = select_best_variant(
    variants_data=variants,
    alpha_prior=1,   # Non-informative prior for fair comparison
    beta_prior=1,
    credible_level=0.95,
    n_simulations=100000
)

# Display results
print("Probability Each Variant is Best")
print("=" * 50)
for name in ['A', 'B', 'C']:
    prob = selection['probabilities'][name]
    bar = '#' * int(prob * 50)
    print(f"  P({name} is best) = {prob:.2%}  {bar}")

winner = selection['best_variant']
print(f"\nWinner: Variant {winner}")
print(f"  Probability of being best: {selection['probabilities'][winner]:.2%}")
print(f"  Posterior mean: {selection['posterior_means'][winner]:.2%}")
ci = selection['credible_intervals'][winner]
print(f"  95% Credible interval: [{ci[0]:.2%}, {ci[1]:.2%}]")
print(f"  Expected loss: {selection['expected_loss'][winner]:.4f}")

# Visualize posterior distributions
posteriors = {}
for name, data in variants.items():
    alpha_post = data['x'] + 1
    beta_post = data['n'] - data['x'] + 1
    posteriors[name] = {
        'alpha': alpha_post,
        'beta': beta_post,
        'mean': alpha_post / (alpha_post + beta_post),
        'ci_95': (beta_dist.ppf(0.025, alpha_post, beta_post),
                  beta_dist.ppf(0.975, alpha_post, beta_post))
    }

fig, ax = plot_multiple_posteriors_comparison(
    posteriors=posteriors,
    control_group_conversion_rate=control_rate,
    epsilon=epsilon
)
plt.show()

# Part 6: The Deployment Bottleneck

## Real-World Timeline
| Date | Event |
|------|-------|
| **Oct** | Bug at launch, variant C gets all the traffic, missing control group |
| **Early Nov** | Variant A wins with >90% probability — we know |
| **Mid Nov** | Amazon release freeze (Black Friday) |
| **Late Nov** | Release issues, more rollbacks |
| **December** | Digital Channel release freeze, holiday delays |
| **January** | Winner (variant A) finally ramped up |

**~3 months** from "we know" to "users benefit" and next iterations


# Part 7: Thompson Sampling

## The Multi-Armed Bandit Problem

<div style="display: flex; align-items: center; gap: 40px;">
<div style="flex: 1;">

**Casino analogy**: K slot machines, unknown payouts, limited budget.

| Casino | A/B Testing |
|--------|-------------|
| Slot machines | Variants |
| Pull lever | Show variant |
| Payout | User converts |

**Dilemma**: Explore (try machines) vs. Exploit (play the best)

</div>
<div style="flex: 0 0 auto; text-align: center;">
<img src="images/multi_armed_bandit.jpg" style="max-height: 320px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.15);">
</div>
</div>

## The Algorithm (5 Lines of Logic)

For each user:

1. **Sample** from each posterior: $\theta_i \sim \text{Beta}(\alpha_i, \beta_i)$
2. **Choose** variant with highest sample
3. **Show** it to the user
4. **Observe** conversion (0 or 1)
5. **Update** posterior: $\alpha \mathrel{+}= r$, $\beta \mathrel{+}= (1-r)$

**Early**: wide posteriors → exploration
**Later**: narrow posteriors → exploitation

In [ ]:
# ┌─────────────────────────────────────────────────────────────────┐
# │          The heart of Thompson Sampling — 5 lines of logic      │
# └─────────────────────────────────────────────────────────────────┘
def thompson_step(alpha, beta, true_rates):
    """One round of Thompson Sampling for one user."""
    samples        = {v: np.random.beta(alpha[v], beta[v]) for v in alpha}  # 1. Sample  θᵢ ~ Beta(αᵢ, βᵢ)
    chosen         = max(samples, key=samples.get)                          # 2. Choose  variant with highest θ
    r              = int(np.random.random() < true_rates[chosen])           # 3+4. Show → observe r ∈ {0, 1}
    alpha[chosen] += r                                                       # 5. Update  α += r
    beta[chosen]  += (1 - r)                                                 # 5. Update  β += 1 − r
    return chosen, r


# ── Simulation harness ────────────────────────────────────────────
def run_thompson_sampling(true_rates, n_users):
    """Run Thompson Sampling over n_users; return traffic, conversions, and history."""
    variants = list(true_rates.keys())
    alpha    = {v: 1 for v in variants}   # Beta(1,1) = uniform prior
    beta     = {v: 1 for v in variants}

    n_shown, n_conv = {v: 0 for v in variants}, {v: 0 for v in variants}
    total_conv = 0
    history = {'user': [], **{f'prob_{v}': [] for v in variants}}

    for uid in range(n_users):
        chosen, r = thompson_step(alpha, beta, true_rates)  # ← the 5 lines above

        n_shown[chosen] += 1
        n_conv[chosen]  += r
        total_conv      += r

        if uid % 50 == 0:   # snapshot: P(each variant is best) via Monte Carlo
            mc     = 10_000
            counts = {v: 0 for v in variants}
            for _ in range(mc):
                s = {v: np.random.beta(alpha[v], beta[v]) for v in variants}
                counts[max(s, key=s.get)] += 1
            history['user'].append(uid)
            for v in variants:
                history[f'prob_{v}'].append(counts[v] / mc)

    return n_shown, n_conv, total_conv, history


def run_fixed_allocation(true_rates, n_users):
    """Baseline: rotate through variants in round-robin order."""
    variants = list(true_rates.keys())
    n_shown, n_conv = {v: 0 for v in variants}, {v: 0 for v in variants}
    total_conv = 0
    for uid in range(n_users):
        chosen     = variants[uid % len(variants)]
        r          = int(np.random.random() < true_rates[chosen])
        n_shown[chosen] += 1
        n_conv[chosen]  += r
        total_conv      += r
    return n_shown, n_conv, total_conv


# ── Run both strategies ───────────────────────────────────────────
np.random.seed(42)
true_rates = {'A': 3244 / 4625, 'B': 1433 / 2100, 'C': 1396 / 2022}
n_users    = 5000

ts_shown, ts_conv, ts_total, ts_history = run_thompson_sampling(true_rates, n_users)
fx_shown, fx_conv, fx_total             = run_fixed_allocation(true_rates, n_users)

best    = max(true_rates, key=true_rates.get)
optimal = n_users * true_rates[best]

# ── Print comparison ──────────────────────────────────────────────
print("THOMPSON SAMPLING vs FIXED ALLOCATION")
print("=" * 60)
print(f"{'':<20s} {'Thompson':>12s} {'Fixed':>12s}")
print("-" * 60)
for v in ['A', 'B', 'C']:
    print(f"Variant {v} traffic:    {100 * ts_shown[v] / n_users:10.1f}%  {100 * fx_shown[v] / n_users:10.1f}%")
print("-" * 60)
print(f"Total conversions:  {ts_total:10d}   {fx_total:10d}")
print(f"Conversion rate:    {100 * ts_total / n_users:10.2f}%  {100 * fx_total / n_users:10.2f}%")
print(f"Regret:             {optimal - ts_total:10.0f}   {optimal - fx_total:10.0f}")
print(f"\nThompson Sampling gained {ts_total - fx_total:.0f} extra conversions")


In [ ]:
# Visualize: P(variant is best) over time
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(ts_history['user'], ts_history['prob_A'], label='P(A is best)', lw=2, color='#2ecc71')
ax.plot(ts_history['user'], ts_history['prob_B'], label='P(B is best)', lw=2, color='#e74c3c')
ax.plot(ts_history['user'], ts_history['prob_C'], label='P(C is best)', lw=2, color='#3498db')
ax.axhline(y=0.95, color='gray', ls='--', lw=1, alpha=0.5, label='95% threshold')
ax.set_xlabel('Number of Users', fontsize=12)
ax.set_ylabel('Probability of Being Best', fontsize=12)
ax.set_title('Thompson Sampling: Learning Which Variant is Best Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='right', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1.05)
plt.tight_layout()
plt.show()

# Find when 95% confidence reached
for i, p in enumerate(ts_history['prob_A']):
    if p >= 0.95:
        print(f"Reached 95% confidence that A is best after ~{ts_history['user'][i]:,} users")
        break
else:
    print(f"Did not reach 95% confidence within {n_users:,} users (but traffic was already optimized)")

## Key Benefits

- **Dynamic traffic allocation** — better variants get more traffic automatically
- **Add variants anytime** — new variants enter with Beta(1,1), compete immediately
- **No stopping rules** — algorithm keeps improving
- **Minimizes regret** — fewer users see inferior variants

## Production Considerations

| Issue | Solution |
|---|---|
| **Delayed feedback** | Batch updates every 10-60 min |
| **Rate drift** | Exponential decay or sliding window |
| **Scalability** | $(\alpha, \beta)$ in distributed cache |
| **Cold start** | Weakly informative priors |

# Summary

| Step | NHST | Bayesian / Automated |
|------|------|---------------------|
| Non-inferiority | "Inconclusive" | "95%+ probability" |
| Best variant | Complex corrections | "A is best at 88%" |
| Deploy winner | Weeks/months | Automatic |
| Add variant | Restart test | Add anytime |

## Bottom Line

1. **Bayesian methods** — work with small samples, actionable probabilities
2. **Thompson Sampling** — automates the entire loop, minimizes regret
3. **Start simple** — Beta-Binomial conjugacy, 5 lines of code

### References

